# Reinforcement Learning

- An agent that 'explores' some space

- As it goes, it learns the value of different state changes in different conditions

- Those values inform subsequente behavior of the agent

## Q-Learning

"Q-learning is a model-free reinforcement learning algorithm to learn the value of an action in a particular state. It does not require a model of the environment (hence "model-free")"

An implementation of reinforement learning

- Set of environmental states $s$

- Set of actions per state $A$. By performing an action ${\displaystyle a\in {A}}$, the agent transitions from state to state.

- Value of each state/action $Q$. (Executing an action in a specific state provides the agent with a reward -a numerical score)

1) Start off with $Q$ values of 0

2) Explore the space

3) When **bad** things happens after a given state/action, it **reduces** its $Q$

4) When **good** things happens after a given state/action, it **increases** its $Q$

"**The goal of the agent is to maximize its total reward**. It does this by adding the maximum reward attainable from future states to the reward for achieving its current state, effectively influencing the current action by the potential future reward. This potential reward is a weighted sum of expected values of the rewards of all future steps starting from the current state"

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSNjoZXhylJVR1BKfio21ntpgj9pUmPcKOFlw&s)

<!---  $Q_t(s,a) = Q_{t-1}(s,a)+\alpha(R(s,a)+\gamma R(s'))$

$Q_t(s,a) = Q_{t-1}(s,a)+\alpha TD_t(a,s)$ --->

$Q(s,a) += \alpha * (R(s,a) + \max(Q(s')) - Q(s,a))$ 

where $s$ is previous state, $s'$ is the current state and $\alpha$ is the discount factor.

new_q = (1-learning_rate) * prev_q + learning_rate * (reward + discount_factor*next_max_q)

# The exploration problem

How do we efficiently explore all of the possible states?

- Simple approach: always choose the action for a given state with the highest Q. If there's a tie, choose at random. (It's inefficient as you might miss a lot of paths)

- Introduce an epsilon term:

    - If a random number is less than $\epsilon$, don't follow the highest Q but choose at random

    - That way, exploration never totally stops

    - But it can be tricky to choose $\epsilon$

**This is Markov Decision Process!**

"A Markov chain is a sequence of states where the probability of moving to the next state depends only on the current state and not on the sequence of events that preceded it."

"In an MDP, an agent makes decisions that influence the transitions between states. Each decision (or action) taken in a particular state leads to a probability distribution over the next possible states, similar to a Markov chain. However, unlike a simple Markov chain, in an MDP, the agent can actively choose actions to optimize a certain objective (usually maximizing some cumulative reward)."

**This is also Dynamic Programming!**

"It refers to simplifying a complicated problem by breaking it down into simpler sub-problems in a recursive manner."

# Q-learning with gym

In [20]:
import gym
import random
import numpy as np

In [3]:
random.seed(1234)

streets = gym.make('Taxi-v3').env
streets.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Actions: 0 = south, 1 = north, 2 = east, 3 = west, 4 = pickup, 5 = dropoff

<img src="https://media.istockphoto.com/id/1151362772/pt/vetorial/diagram-compass-rose-for-navigation-orientation.jpg?s=612x612&w=0&k=20&c=xICp8-cqYnvPir4UB54lvkqIfrDcScQKtjeo79dGnjE=" alt="Drawing" style="width: 200px;"/>

Defining initial state at (2,3)

In [34]:
#(2 - axis X,
# 3 - axis Y,
# 2 - where pick up passenger,
# 0 - destination)
initial_state = streets.encode(2,3,2,0)
streets.s = initial_state
streets.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


In [35]:
#Reward table for each action (0 = south, 1 = north, 2 = east, 3 = west, 4 = pickup, 5 = dropoff)
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

action: [(**prob. assign** to that action, **next state result** for that action, **reward** for that action, sucessful **to dropoff**)]

Q-learning

In [36]:
q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

#hyperparameters
learning_rate = 0.1
discount_factor = 0.6
exploration = 0.1
epochs = 10000

for taxi_run in range(epochs):
    state = streets.reset()
    done = False

    while not done:
        random_value = random.uniform(0,1)
        if (random_value < exploration):
            action = streets.action_space.sample() #explore a random action
        else:
            action = np.argmax(q_table[state])
        
        next_state, reward, done, info = streets.step(action)

        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1-learning_rate) * prev_q + learning_rate * (reward + discount_factor*next_max_q)
        q_table[state,action] = new_q

        state = next_state

In [37]:
q_table[initial_state]

array([-2.40128866, -2.39738234, -2.40248868, -2.3639511 , -6.33514305,
       -6.86827889])

In [38]:
from IPython.display import clear_output
from time import sleep

for tripnum in range (1,11):
    state = streets.reset()

    done= False

    while not done:
        action = np.argmax(q_table[state])
        next_state, reward, done, info = streets.step(action)
        clear_output(wait=True)
        print("Trip number "+str(tripnum))
        print(streets.render(mode='ansi'))
        sleep(.5)
        state=next_state
    sleep(2)


Trip number 10
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

